In [ ]:
import pyproj
geodesic = pyproj.Geod(ellps='WGS84')

In [ ]:
start = (28.602104, -81.196870)



In [ ]:
import json
import math

with open("/Users/owenburns/workareas/cavrel-platooning/missions/tracks/running_track.json", 'r') as f:
    track = json.load(f)

def calculate_straight(start, end, seg_dist):
    bearing, _, dist = geodesic.inv(start[1], start[0], end[1], end[0])
    seg_count = int(dist/seg_dist)
    segs = [(start[0], start[1], bearing)]
    for _ in range(seg_count):
        lat_prev, lon_prev, _ = segs[-1]
        lon_next, lat_next, _ = geodesic.fwd(lon_prev, lat_prev, bearing, seg_dist)
        segs.append((lat_next, lon_next, bearing))

    if abs(bearing) < 1e-3:
        bearing = 360.0

    return segs, bearing

def calculate_turn(start, start_bearing, end, end_bearing, seg_dist, direction='left'):
    _, _, dist = geodesic.inv(start[1], start[0], end[1], end[0])
    arclen = math.pi*(dist/2)
    seg_count = int(arclen/seg_dist)
    bearing_diff_roc = abs((end_bearing - start_bearing)/seg_count)

    if direction == 'right':
        bearing_diff_roc *= -1

    segs = [(start[0], start[1], start_bearing)]
    for _ in range(seg_count):
        lat_prev, lon_prev, bearing_prev = segs[-1]
        bearing_next = bearing_prev - bearing_diff_roc
        lon_next, lat_next, _ = geodesic.fwd(lon_prev, lat_prev, bearing_next, seg_dist)
        segs.append((lat_next, lon_next, bearing_next))
    return segs

def calculate_track(straights, turns, broadcast_interval):
    straight_points = []
    bearings = []
    for straight in straights:
        seg_dist = broadcast_interval*straight['speed']
        start = (straight['start']['lat'], straight['start']['lon'])
        end = (straight['end']['lat'], straight['end']['lon'])
        segs, bearing = calculate_straight(start, end, seg_dist)
        straight_points.append(segs)
        bearings.append(bearing)

    bearings = [(bearings[i], bearings[(i+1)%len(bearings)]) for i in range(len(bearings))]
    turn_points = []
    for turn, (start_bearing, end_bearing) in zip(turns, bearings):
        seg_dist = broadcast_interval*turn['speed']
        start = (turn['start']['lat'], turn['start']['lon'])
        end = (turn['end']['lat'], turn['end']['lon'])
        segs = calculate_turn(start, start_bearing, end, end_bearing, seg_dist, turn['direction'])
        turn_points.append(segs)

    full_track = []
    for straight, turn in zip(straight_points, turn_points):
        full_track += straight + turn

    return full_track

full_track = calculate_track(track['straights'], track['turns'], 0.1)

In [ ]:
from typing import List
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.animation import FuncAnimation
from matplotlib.text import Annotation
from matplotlib.lines import Line2D
from IPython.display import HTML
import matplotlib
import math
matplotlib.rcParams['animation.embed_limit'] = 2**128
EARTH_RADIUS = 6371e3
DUE_EAST = 90

center_latitude = track['center']['lat']
center_longitude = track['center']['lon']
center_orientation = DUE_EAST

def coords_to_local(target_lat, target_lon):
    # Convert lat, lon to radians
    target_lat_rad, target_lon_rad = math.radians(target_lat), math.radians(target_lon)
    current_lat_rad, current_lon_rad = math.radians(center_latitude), math.radians(center_longitude)

    x = EARTH_RADIUS * (target_lon_rad - current_lon_rad) * math.cos((current_lat_rad + target_lat_rad) / 2)
    y = EARTH_RADIUS * (target_lat_rad - current_lat_rad)

    angle = math.radians(center_orientation - DUE_EAST)
    qx = math.cos(angle) * x - math.sin(angle) * y
    qy = math.sin(angle) * x + math.cos(angle) * y
    return qx, qy

xy_points = [coords_to_local(lat, lon) for lat, lon, heading in full_track]
xs = [point[0] for point in xy_points]
ys = [point[1] for point in xy_points]

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(autoscale_on=False, xlim=(-100, 100), ylim=(-100, 100))
ax.plot(xs, ys, 'k-', label='Track Path')  # green dots connected by lines
ax.plot(xs[0], ys[0], 'yo', label='Start/End Point')
ax.set_aspect('equal')

pt = Line2D([xs[0]], [ys[0]], marker='o', color='green')

def init_func() -> list:
    print('init_func')
    ax.add_artist(pt)
    return []

def func(frame, *fargs) -> list:
    global pt

    pt.set_data([xs[frame]], [ys[frame]])

    return []

ax.set_xlabel('Meters East of Center')
ax.set_ylabel('Meters North of Center')
ax.set_title('Vehicle Path and Vectors')
ax.legend()
ax.grid()

# Create animation
ani = FuncAnimation(fig, func, range(len(xy_points)), init_func, blit=True, interval=50) # Adjust interval as needed
HTML(ani.to_jshtml())